In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
state = AppState()
config = {}
trading_historical_data = {}
account = None
account = TradingAccount(app)
state.last_action = "SELL"
dataframe_dict = []
pd.set_option('display.max_columns', None)

In [2]:
def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])

    action = '' 
#     print(df_last['open'], df_last['close'], df_last['high'], df_last['low'])
    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':
        
        action = 'BUY'

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    if len(df) == 0:
        return df
    return df.iloc[iterations - 1:iterations]


In [3]:
def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [78]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    ta = TechnicalAnalysis(trading_dataCopy)
    ta.addAll()
    df = ta.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            # technical indicators
            ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
            ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
            goldencross = bool(df_last['goldencross'].values[0])
            macdgtsignal = bool(df_last['macdgtsignal'].values[0])
            macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
            ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
            ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
            macdltsignal = bool(df_last['macdltsignal'].values[0])
            macdltsignalco = bool(df_last['macdltsignalco'].values[0])
            obv = float(df_last['obv'].values[0])
            obv_pc = float(df_last['obv_pc'].values[0])
            elder_ray_buy = bool(df_last['eri_buy'].values[0])
            elder_ray_sell = bool(df_last['eri_sell'].values[0])
            rri = df_last['rri'].values[0]
            last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
            last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
            last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
            last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
            rri_buy = df_last['rri_buy'].values[0]
            rri_sell = df_last['rri_sell'].values[0]
            
            hammer = bool(df_last['hammer'].values[0])
            inverted_hammer = bool(df_last['inverted_hammer'].values[0])
            hanging_man = bool(df_last['hanging_man'].values[0])
            shooting_star = bool(df_last['shooting_star'].values[0])
            three_white_soldiers = bool(df_last['three_white_soldiers'].values[0])
            three_black_crows = bool(df_last['three_black_crows'].values[0])
            morning_star = bool(df_last['morning_star'].values[0])
            evening_star = bool(df_last['evening_star'].values[0])
            three_line_strike = bool(df_last['three_line_strike'].values[0])
            abandoned_baby = bool(df_last['abandoned_baby'].values[0])
            morning_doji_star = bool(df_last['morning_doji_star'].values[0])
            evening_doji_star = bool(df_last['evening_doji_star'].values[0])
            two_black_gapping = bool(df_last['two_black_gapping'].values[0])
            
            last_3_hammer = bool(df_last['hammer'].values[0]) or bool(df_last['hammer'].shift(1).values[0]) or bool(df_last['hammer'].shift(2).values[0])
            last_3_inverted_hammer = bool(df_last['inverted_hammer'].values[0]) or bool(df_last['inverted_hammer'].shift(1).values[0]) or bool(df_last['inverted_hammer'].shift(2).values[0])
            last_3_hanging_man = bool(df_last['hanging_man'].values[0]) or bool(df_last['hanging_man'].shift(1).values[0]) or bool(df_last['hanging_man'].shift(2).values[0])
            last_3_shooting_star = bool(df_last['shooting_star'].values[0]) or bool(df_last['shooting_star'].shift(1).values[0]) or bool(df_last['shooting_star'].shift(2).values[0])
            last_3_three_white_soldiers = bool(df_last['three_white_soldiers'].values[0]) or bool(df_last['three_white_soldiers'].shift(1).values[0]) or bool(df_last['three_white_soldiers'].shift(2).values[0])
            last_3_three_black_crows = bool(df_last['three_black_crows'].values[0]) or bool(df_last['three_black_crows'].shift(1).values[0]) or bool(df_last['three_black_crows'].shift(2).values[0])
            last_3_morning_star = bool(df_last['morning_star'].values[0]) or bool(df_last['morning_star'].shift(1).values[0]) or bool(df_last['morning_star'].shift(2).values[0])
            last_3_evening_star = bool(df_last['evening_star'].values[0]) or bool(df_last['evening_star'].shift(1).values[0]) or bool(df_last['evening_star'].shift(2).values[0])
            last_3_three_line_strike = bool(df_last['three_line_strike'].values[0]) or bool(df_last['three_line_strike'].shift(1).values[0]) or bool(df_last['three_line_strike'].shift(2).values[0])
            last_3_abandoned_baby = bool(df_last['abandoned_baby'].values[0]) or bool(df_last['abandoned_baby'].shift(1).values[0]) or bool(df_last['abandoned_baby'].shift(2).values[0])
            last_3_morning_doji_star = bool(df_last['morning_doji_star'].values[0]) or bool(df_last['morning_doji_star'].shift(1).values[0]) or bool(df_last['morning_doji_star'].shift(2).values[0])
            last_3_evening_doji_star = bool(df_last['evening_doji_star'].values[0]) or bool(df_last['evening_doji_star'].shift(1).values[0]) or bool(df_last['evening_doji_star'].shift(2).values[0])
            last_3_two_black_gapping = bool(df_last['two_black_gapping'].values[0]) or bool(df_last['two_black_gapping'].shift(1).values[0]) or bool(df_last['two_black_gapping'].shift(2).values[0])
            
            print(last_3_hammer, last_3_inverted_hammer, last_3_hanging_man, last_3_shooting_star, last_3_three_white_soldiers, last_3_three_black_crows)
            
            # if simulation interations < 200 set goldencross to true
            if app.isSimulation() and state.iterations < 200:
                goldencross = True
            buy_score = 0
            if ema12gtema26: buy_score += 3
            if ema12gtema26co: buy_score += 3
            if macdgtsignal: buy_score += 3
            if goldencross: buy_score += 2
            if macdgtsignalco: buy_score +=3
            if obv_pc > -5: buy_score += 3
            if elder_ray_buy: buy_score += 2
            if not elder_ray_sell: buy_score += 2
            if rri_buy and (ema12gtema26 or ema12gtema26co) and (ema12gtema26co or macdgtsignalco):
                buy_score += 10
            elif rri_buy and (ema12gtema26 and macdgtsignal):
                buy_score += 8
            elif rri_buy and (ema12gtema26 or macdgtsignal):
                buy_score += 6
            elif rri_buy:
                buy_score += 4
            candle_stick_score = 0
            # weak up
            if hammer is True: candle_stick_score += 1
            if inverted_hammer is True: candle_stick_score += 1
            # weak down
            if shooting_star is True: candle_stick_score -= 1
            if hanging_man is True: candle_stick_score -= 1
            # strong up
            if three_white_soldiers is True: candle_stick_score += 2
            if morning_star is True: candle_stick_score += 2
            # strong down
            if three_black_crows is True: candle_stick_score -= 2
            if evening_star is True: candle_stick_score -= 2
            # reliable up
            if three_line_strike is True: candle_stick_score += 3
            if abandoned_baby is True: candle_stick_score += 3
            if morning_doji_star is True: candle_stick_score += 3
            # reliable down
            if evening_doji_star is True: candle_stick_score -= 3
            if two_black_gapping is True: candle_stick_score -= 3
            
            last_3_candle_stick_score = 0  
            # weak up
            if last_3_hammer is True: last_3_candle_stick_score += 1
            if last_3_inverted_hammer is True: last_3_candle_stick_score += 1
            # weak down
            if last_3_shooting_star is True: last_3_candle_stick_score -= 1
            if last_3_hanging_man is True: last_3_candle_stick_score -= 1
            # strong up
            if last_3_three_white_soldiers is True: last_3_candle_stick_score += 2
            if last_3_morning_star is True: last_3_candle_stick_score += 2
            # strong down
            if last_3_three_black_crows is True: last_3_candle_stick_score -= 2
            if last_3_evening_star is True: last_3_candle_stick_score -= 2
            # reliable up
            if last_3_three_line_strike is True: last_3_candle_stick_score += 3
            if last_3_abandoned_baby is True: last_3_candle_stick_score += 3
            if last_3_morning_doji_star is True: last_3_candle_stick_score += 3
            # reliable down
            if last_3_evening_doji_star is True: last_3_candle_stick_score -= 3
            if last_3_two_black_gapping is True: last_3_candle_stick_score -= 3
                
            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append([state.action, buy_score, candle_stick_score,last_3_candle_stick_score, app.getGranularity(), price, ema12gtema26, ema12gtema26co,
                                     goldencross, macdgtsignal, macdgtsignalco, obv, obv_pc, elder_ray_buy,
                                     elder_ray_sell, rri, rri_buy, rri_sell, last_3_rri_buy, last_5_rri_buy, last_3_rri_sell, last_5_rri_sell, one_hr_high, one_hr_low, three_hr_high, three_hr_low, 
                                     three_six_high,three_six_low, six_12_high, six_12_low, six_hr_high, six_hr_low,
                                     nine_hr_high, nine_hr_low, twelve_hr_high, twelve_hr_low])
    

In [ ]:
%%time
app = PyCryptoBot()
state = AppState()
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
                          'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
                          'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
                          'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
                          'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
                          'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
                          'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
                          'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
                          'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
                          'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
                          'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
                          'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
                          'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
                          'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
                       'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
                       'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
                       'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
                       'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
                       'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
                       'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
                       'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
                       'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
                       'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
                       'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
                       'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
                       'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
                       'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
                       'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
                       'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
                       'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
                       'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
                       'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
                       'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
                       'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
                       'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
                       'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
                       'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
                       'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
                        'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
                       'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
                       'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
                        'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
                       'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
                       'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
                       'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
                       'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
                       'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
                       'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
                       'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
                       'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
                       'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
                       'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
                       'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
                       'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
                       'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
                       'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
                       'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
                       'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
                       'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
                       'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
                       'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
                       'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
                       'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
                       'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
                       'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
                       'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
                       'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
                       'MDXBUSD', 'MASKBUSD']
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict,
                         columns=['action', 'buy_score', 'candle_stick_score', 'last_3_candle_stick_score', 'Granularity', 'Price', 'Fast EMA gt', 'EMA CO',
                                  'Golden Cross', 'Macdgtsignal', 'Macdgtsignalco', 'Obv', 'Obv_pc',
                                  'Elder_ray_buy', 'elder_ray_sell', 'rri', 'rri_buy', 'rri_sell', 'last_3_rri_buy', 'last_5_rri_buy','last_3_rri_sell', 'last_5_rri_sell', 'one_hr_high', 'one_hr_low', 'three_hr_high',
                                  'three_hr_low', 'three_six_high','three_six_low', 'six_12_high', 'six_12_low',
                                  'six_hr_high', 'six_hr_low', 'nine_hr_high', 'nine_hr_low',
                                  'twelve_hr_high', 'twelve_hr_low'])


calculating  WTCBNB
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
calculating  NEOBNB
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

...

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True 

In [77]:
buy_dataframe.describe()

,buy_score,candle_stick_score,last_3_candle_stick_score,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,16547.000000,16547.000000,16547.0,16547.0,16547.000000,1.654700e+04,16547.000000,16538.000000,16536.000000,16536.000000,16536.000000,16536.000000,16223.000000,16223.000000,16118.000000,16118.000000,16536.000000,16536.000000,16536.000000,16536.000000,16536.000000,16536.000000
mean,12.764489,-0.006648,3.0,3600.0,307.426560,-6.747333e+07,-21.022565,-0.534254,1.428929,-1.328053,2.297352,-2.379841,3.059049,-3.871524,4.180813,-5.514434,3.275429,-3.494545,3.991435,-4.397021,4.615116,-5.097348
std,3.872291,0.344113,0.0,0.0,3186.279661,1.378397e+09,3283.247675,0.603405,2.038011,1.452918,2.939940,2.336105,5.736092,4.380565,9.066384,6.124871,4.108889,3.175353,5.409416,3.890295,7.619648,4.487405
min,4.000000,-5.000000,3.0,3600.0,0.000100,-6.486502e+10,-409535.520000,-1.000000,-1.750000,-24.010000,-1.750000,-24.640000,-12.690000,-78.180000,-14.350000,-78.740000,-1.410000,-24.640000,-1.410000,-78.180000,-1.410000,-78.740000
25%,10.000000,0.000000,3.0,3600.0,0.066639,-6.154628e+05,-5.980000,-0.893771,0.460000,-1.860000,0.730000,-3.280000,0.240000,-6.210000,0.230000,-8.950000,1.020000,-4.920000,1.210000,-6.390000,1.350000,-7.442500
50%,13.000000,0.000000,3.0,3600.0,0.820500,-3.106335e+03,0.980000,-0.811282,0.980000,-0.980000,1.550000,-1.720000,2.180000,-2.990000,2.950000,-4.290000,2.260000,-2.520000,2.700000,-3.210000,3.090000,-3.720000
75%,16.000000,0.000000,3.0,3600.0,6.939200,2.728842e+05,9.850000,-0.671992,1.810000,-0.370000,2.970000,-0.750000,4.680000,-0.960000,6.360000,-1.340000,4.242500,-1.210000,5.030000,-1.510000,5.630000,-1.750000
max,28.000000,5.000000,3.0,3600.0,47553.470000,6.423951e+09,42515.660000,1.000000,140.260000,2.110000,140.260000,1.870000,185.200000,21.390000,397.840000,38.530000,140.260000,1.830000,185.200000,1.830000,397.840000,1.720000


In [73]:
buy_dataframe.query("candle_stick_score>4")

,action,buy_score,candle_stick_score,Granularity,Price,Fast EMA gt,EMA CO,Golden Cross,Macdgtsignal,Macdgtsignalco,Obv,Obv_pc,Elder_ray_buy,elder_ray_sell,rri,rri_buy,rri_sell,last_3_rri_buy,last_5_rri_buy,last_3_rri_sell,last_5_rri_sell,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
2479,BUY,13,5,3600,0.00163,True,True,False,True,False,-27456.5,-7.86,True,False,0.714286,False,False,False,True,False,False,-0.12,-0.61,1.23,-0.98,3.13,-0.98,3.13,-0.06,1.23,-0.98,3.13,-0.98,3.13,-0.98


### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour



In [1]:
rri_buys = buy_dataframe.query("rri_buy==True")
rri_buys.describe()

NameError: name 'buy_dataframe' is not defined

In [8]:
non_rri_buys = buy_dataframe.query("rri_buy==False")
non_rri_buys.describe()

,buy_score,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,4174.000000,4174.0,4174.000000,4.174000e+03,4174.000000,4165.000000,4171.000000,4171.000000,4171.000000,4171.000000,4160.000000,4160.000000,4136.000000,4136.000000,4171.000000,4171.000000,4171.000000,4171.000000,4171.000000,4171.000000
mean,14.694058,3600.0,352.479598,-7.565683e+07,2.876605,0.441675,1.425780,-1.482103,2.327662,-2.666320,2.656029,-4.201022,3.498561,-6.432909,3.198784,-3.675936,3.913522,-4.629890,4.509501,-5.467909
std,2.555884,0.0,3445.753363,1.268248e+09,1137.688975,0.392471,3.079633,1.571487,4.014430,2.409705,6.143330,4.379574,9.965331,6.834598,4.910480,3.098810,6.091822,3.902179,8.963709,4.660950
min,9.000000,3600.0,0.000104,-4.272554e+10,-47075.060000,-1.000000,-1.630000,-24.010000,-1.630000,-24.640000,-12.430000,-25.520000,-14.350000,-35.820000,-1.410000,-24.640000,-1.410000,-25.520000,-1.410000,-35.820000
25%,13.000000,3600.0,0.082983,-3.717059e+05,-8.807500,0.235294,0.310000,-1.950000,0.480000,-3.560000,-0.270000,-6.230000,-0.560000,-10.082500,0.690000,-4.945000,0.860000,-6.285000,0.980000,-7.610000
50%,15.000000,3600.0,0.944350,3.801183e+03,0.855000,0.526316,0.790000,-1.100000,1.290000,-2.020000,1.600000,-3.410000,2.180000,-4.890000,1.860000,-2.900000,2.290000,-3.540000,2.570000,-4.020000
75%,16.000000,3600.0,7.334750,6.221400e+05,17.650000,0.735280,1.670000,-0.520000,2.835000,-1.010000,4.290000,-1.450000,5.740000,-1.715000,4.010000,-1.520000,4.930000,-1.920000,5.460000,-2.070000
max,21.000000,3600.0,47553.470000,6.423951e+09,39021.950000,1.000000,140.260000,1.690000,140.260000,1.690000,107.720000,19.340000,394.220000,21.420000,140.260000,1.690000,140.260000,1.690000,394.220000,1.690000


In [9]:
loss_buys = buy_dataframe.query("three_hr_high<1.1").query("three_six_high<0.05").query("six_12_high<-5")
loss_buys.describe()

,buy_score,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,530.000000,530.0,530.000000,5.300000e+02,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000,530.000000
mean,12.964151,3600.0,45.845636,5.610708e+06,-4.702943,-0.436445,0.320604,-3.324472,0.337019,-6.375453,-3.648736,-13.095038,-7.099094,-15.588321,0.337019,-10.732660,0.337019,-13.106038,0.337019,-14.475472
std,3.686259,0.0,266.426514,2.965075e+08,512.365462,0.637351,0.359015,2.498378,0.358696,3.057568,2.174875,3.119478,1.712735,3.475757,0.358696,3.001805,0.358696,3.139986,0.358696,3.239241
min,4.000000,3600.0,0.000162,-4.623721e+09,-10670.860000,-1.000000,-0.890000,-24.010000,-0.890000,-24.640000,-12.430000,-24.970000,-14.350000,-28.310000,-0.890000,-24.640000,-0.890000,-24.970000,-0.890000,-25.490000
25%,11.000000,3600.0,0.202843,-1.897523e+05,-8.832500,-0.863245,0.030000,-4.805000,0.040000,-7.945000,-4.915000,-14.947500,-8.070000,-17.565000,0.040000,-12.077500,0.040000,-14.947500,0.040000,-16.512500
50%,14.000000,3600.0,1.056450,8.577003e+04,-1.035000,-0.791475,0.300000,-2.635000,0.310000,-6.105000,-3.430000,-12.470000,-6.645000,-15.340000,0.310000,-10.150000,0.310000,-12.470000,0.310000,-14.150000
75%,16.000000,3600.0,7.324000,2.823933e+06,8.727500,0.209913,0.580000,-1.530000,0.600000,-4.107500,-2.142500,-10.795000,-5.775000,-13.097500,0.600000,-8.710000,0.600000,-10.795000,0.600000,-12.080000
max,21.000000,3600.0,3658.890000,3.543153e+09,3574.550000,1.000000,1.090000,0.040000,1.090000,-1.260000,0.000000,-7.310000,-5.010000,-7.540000,1.090000,-5.180000,1.090000,-7.310000,1.090000,-7.540000


In [10]:
loss_buys_with_sell= loss_buys[loss_buys['Obv_pc'] < -5]
loss_buys_with_sell.describe()

,buy_score,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,180.000000,180.0,180.000000,1.800000e+02,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,11.011111,3600.0,56.360786,-1.681758e+06,-105.096556,-0.515325,0.359278,-3.419944,0.369611,-6.527444,-3.756889,-13.174167,-6.925222,-15.577667,0.369611,-10.912556,0.369611,-13.176889,0.369611,-14.423167
std,3.390735,0.0,314.514573,1.236467e+08,796.083680,0.585787,0.352209,2.398055,0.358269,2.896168,2.090741,3.226047,1.659480,3.484172,0.358269,3.082599,0.358269,3.230278,0.358269,3.361691
min,4.000000,3600.0,0.000162,-1.180762e+09,-10670.860000,-1.000000,-0.620000,-18.410000,-0.620000,-18.410000,-12.160000,-24.970000,-12.130000,-25.860000,-0.620000,-20.810000,-0.620000,-24.970000,-0.620000,-25.490000
25%,9.000000,3600.0,0.186215,2.979773e+03,-43.430000,-0.866024,0.070000,-5.152500,0.070000,-8.092500,-4.975000,-15.347500,-7.702500,-17.415000,0.070000,-12.455000,0.070000,-15.347500,0.070000,-16.535000
50%,11.000000,3600.0,0.917650,2.079179e+05,-15.970000,-0.801727,0.325000,-2.930000,0.335000,-6.290000,-3.480000,-12.510000,-6.380000,-15.670000,0.335000,-10.235000,0.335000,-12.510000,0.335000,-14.170000
75%,13.000000,3600.0,4.454125,2.609465e+06,-8.717500,-0.268850,0.630000,-1.527500,0.640000,-4.470000,-2.282500,-10.755000,-5.690000,-12.885000,0.640000,-8.595000,0.640000,-10.755000,0.640000,-11.937500
max,16.000000,3600.0,3657.610000,3.878650e+08,-5.040000,0.930818,1.080000,-0.020000,1.080000,-2.090000,0.000000,-7.540000,-5.030000,-7.540000,1.080000,-5.530000,1.080000,-7.540000,1.080000,-7.540000


In [11]:
df_last['rri']

NameError: name 'df_last' is not defined

In [ ]:
trading_historical_data

In [ ]:
df_last['rri']

In [74]:
last_3_hammer = bool(df_last['hammer'].values[0]) or bool(df_last['hammer'].shift(1).values[0]) or bool(df_last['hammer'].shift(2).values[0])

NameError: name 'df_last' is not defined